In [1]:
pwd

'/home/wsuser/work'

# Load The Image Dataset

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_8f7148c95aec46a58ecaef6266303206 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='W2QPUKt4Q6wipECq06xngbphpqZemUtU0VpPpeJ2WuHQ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_8f7148c95aec46a58ecaef6266303206.get_object(Bucket='vegtablek-donotdelete-pr-ysxrxws3vxpfxq', Key='Dataset Plant Disease.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


# UnZip The Folder

In [3]:
from io import BytesIO
import zipfile 
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
ls

'Dataset Plant Disease'/   vegetable.h5   vegetable-training.tgz


In [5]:
pwd

'/home/wsuser/work'

# Image Augmentation

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
ls

'Dataset Plant Disease'/   vegetable.h5   vegetable-training.tgz


In [9]:
pwd

'/home/wsuser/work'

In [10]:
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/train_set",target_size = (128,128),batch_size=16,class_mode = 'categorical')
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/test_set",target_size = (128,128),batch_size=16,class_mode = 'categorical')

Found 11386 images belonging to 9 classes.
Found 3416 images belonging to 9 classes.


In [11]:
x_train.class_indices

{'Pepper,_bell___Bacterial_spot': 0,
 'Pepper,_bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___Late_blight': 3,
 'Potato___healthy': 4,
 'Tomato___Bacterial_spot': 5,
 'Tomato___Late_blight': 6,
 'Tomato___Leaf_Mold': 7,
 'Tomato___Septoria_leaf_spot': 8}

# CNN

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
model = Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Hidden Layers

In [13]:
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))
model.add(Dense(75,activation='relu'))

In [14]:
model.add(Dense(9,activation = 'softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 300)               38102700  
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 75)                11325     
                                                        

In [ ]:
model.fit_generator(x_train,steps_per_epoch=89,epochs=20,validation_data=x_test,validation_steps=27)

/tmp/wsuser/ipykernel_4106/174847055.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=89,epochs=20,validation_data=x_test,validation_steps=27)


Epoch 1/20
89/89 [==============================] - 46s 504ms/step - loss: 2.7364 - accuracy: 0.2507 - val_loss: 1.6110 - val_accuracy: 0.4051
Epoch 2/20
89/89 [==============================] - 44s 489ms/step - loss: 1.5012 - accuracy: 0.4775 - val_loss: 1.2679 - val_accuracy: 0.5532
Epoch 3/20
89/89 [==============================] - 44s 494ms/step - loss: 1.1257 - accuracy: 0.6096 - val_loss: 1.0216 - val_accuracy: 0.6481
Epoch 4/20
89/89 [==============================] - 43s 486ms/step - loss: 1.0272 - accuracy: 0.6375 - val_loss: 1.2224 - val_accuracy: 0.5671
Epoch 5/20
89/89 [==============================] - 44s 489ms/step - loss: 0.9516 - accuracy: 0.6397 - val_loss: 0.7708 - val_accuracy: 0.7292
Epoch 6/20
89/89 [==============================] - 43s 483ms/step - loss: 0.8239 - accuracy: 0.7072 - val_loss: 0.6976 - val_accuracy: 0.7523
Epoch 7/20
89/89 [==============================] - 44s 491ms/step - loss: 0.7302 - accuracy: 0.7395 - val_loss: 0.7749 - val_accuracy: 0.7500

# Saving The Model

In [ ]:
model.save("vegetable.h5")

In [ ]:
ls

# Testing The Model

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
#Load the model

model =load_model("vegetable.h5")

In [ ]:
img = image.load_img(r"/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/train_set/Pepper,_bell___healthy/0a3f2927-4410-46a3-bfda-5f4769a5aaf8___JR_HL 8275.JPG",target_size=(128,128))
img

In [ ]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)

index=['Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot']

index[pred[0]]

In [ ]:
img = image.load_img(r"/home/wsuser/work/Dataset Plant Disease/Veg-dataset/Veg-dataset/train_set/Tomato___Septoria_leaf_spot/00f16858-f392-4d9e-ad9f-efab8049a13f___JR_Sept.L.S 8368.JPG",target_size=(128,128))
img

In [ ]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)

index=['Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot']

index[pred[0]]

# IBM Deployment 

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials= {
                    "url" : "https://us-south.ml.cloud.ibm.com",
                    "apikey" : "QN8ZXpjqYCXQSwOqftU0cxHgzRIuDTPxhrb9QWTGmDt2"
                }

In [ ]:
client = APIClient(wml_credentials)


In [ ]:
client

In [ ]:
def guid_from_space_name(client,Vegetable_deploy):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == Vegetable_deploy)['metadata']['id'])


In [ ]:
space_uid = guid_from_space_name(client,'Vegetable_deploy')
print("space UID = " + space_uid)


In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
#tensorflow_1.15-py3.6
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_space_uid

In [ ]:
ls

In [ ]:
!tar -zcvf vegetable-training.tgz vegetable.h5

In [ ]:
ls


In [ ]:
model_details = client.repository.store_model(model= 'vegetable-training.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
    )

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

In [ ]:
lient.repository.download(model_id,'vegetable.tar.gb')

In [ ]:
ls